In [1]:
from pyspark.sql import SparkSession
import numpy as py
from pyspark.sql.types import Row
import pyspark.sql.functions as fn 


In [2]:
spark = SparkSession     .builder     .config("spark.driver.memory", "15g") .config("spark.executor.memory", "15g") .appName("PySpark assignment")     .getOrCreate()



In [3]:
r = spark.sparkContext.parallelize(range(10000))
r = spark.sparkContext.textFile("../terraform/gutenberg/*.txt")



In [4]:
rdd = spark.read.json('dataset_en_train.json').rdd



In [5]:
def lower_clean_str(x):
  punc='!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
  lowercased_str = x.lower()
  for ch in punc:
    lowercased_str = lowercased_str.replace(ch, '')
  return lowercased_str

def parse(rdd):
    # Do some Python to turn our PGN into a Row
    # Regular expression 
    string = rdd
    # note: regular expression '.*' -> any thing for any number of times
    #f = re.search(r'*',string)
    return Row(doc = rdd) 


In [6]:

def reviewRDD_to_DF(rdd):
  reviewListRDD = rdd.flatMap(lambda x: x.review_body.lower().splitlines() + 
                          x.review_title.lower().splitlines())
  reviewListRDD = reviewListRDD.map(lower_clean_str)
  reviewListDF = reviewListRDD.map(parse).toDF()
  return reviewListDF


In [7]:
from nltk import ngrams
from pyspark.sql.functions import split
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.ml.feature import NGram
from pyspark.sql.functions import explode

def nGram_DF_Creating(ngram,reviewListDF):
  bnGramDF = reviewListDF.select('doc', split(reviewListDF.doc, " ").alias("feature"))
  if ngram >1:
    nGramDF = NGram(n=ngram,inputCol = 'feature',outputCol='grams').transform(nGramDF)
    nGramDF = nGramDF.drop('doc')
    nGramDF = nGramDF.withColumn("doc_id", monotonically_increasing_id())
    nGramDF = nGramDF.withColumn("token", explode(nGramDF.grams))
  else:
    nGramDF = nGramDF.drop('doc')
    nGramDF = nGramDF.withColumn("doc_id", monotonically_increasing_id())
    nGramDF = nGramDF.withColumn("token", explode(nGramDF.feature))
  return nGramDF




In [8]:

import numpy as np
from pyspark.sql.functions import col
from pyspark.sql.functions import log

def dt_idf_DF_Create(input_df,totalcount):

    input_df.createOrReplaceTempView("input_df")
    tfreq = spark.sql("SELECT doc_id,token,COUNT(feature) AS tf FROM input_df GROUP BY doc_id, token")
    dfreq = spark.sql("SELECT token,COUNT(DISTINCT feature) AS dfcount FROM input_df GROUP BY token")
 

    dfreq = dfreq.withColumn("idf",log(totalcount/(dfreq.dfcount+1)))
    totalcount = reviewListDF.count()
    dfreq = dfreq.withColumn("idf",log(totalcount/(dfreq.dfcount+1)))


    tfreq.createOrReplaceTempView("tfreq")
    dfreq.createOrReplaceTempView("dfreq")
    tf_idf = spark.sql("SELECT tfreq.token, tfreq.doc_id, tfreq.tf,dfreq.dfcount ,dfreq.idf FROM dfreq LEFT JOIN tfreq ON tfreq.token = dfreq.token")
    tf_idf = tf_idf.withColumn('tf_idf_stats',tf_idf.tf * tf_idf.idf)

    return tf_idf






In [9]:
reviewDF = reviewRDD_to_DF(rdd)


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 1.0 failed 1 times, most recent failure: Lost task 0.0 in stage 1.0 (TID 8, 192.168.0.21, executor driver): java.net.SocketException: Software caused connection abort: socket write error
	at java.base/java.net.SocketOutputStream.socketWrite0(Native Method)
	at java.base/java.net.SocketOutputStream.socketWrite(SocketOutputStream.java:110)
	at java.base/java.net.SocketOutputStream.write(SocketOutputStream.java:150)
	at java.base/java.io.BufferedOutputStream.write(BufferedOutputStream.java:123)
	at java.base/java.io.DataOutputStream.write(DataOutputStream.java:107)
	at java.base/java.io.FilterOutputStream.write(FilterOutputStream.java:108)
	at org.apache.spark.api.python.PythonRDD$.write$1(PythonRDD.scala:283)
	at org.apache.spark.api.python.PythonRDD$.$anonfun$writeIteratorToStream$1(PythonRDD.scala:295

In [10]:
reviewDF = reviewRDD_to_DF(rdd)


In [11]:
monoGramDF = nGram_DF_Creating(1,reviewDF)


UnboundLocalError: local variable 'nGramDF' referenced before assignment

In [12]:
monoGramDF = nGram_DF_Creating(1,reviewDF)


UnboundLocalError: local variable 'nGramDF' referenced before assignment

In [13]:
from nltk import ngrams
from pyspark.sql.functions import split
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.ml.feature import NGram
from pyspark.sql.functions import explode

def nGram_DF_Creating(ngram,reviewListDF):
    nGramDF = reviewListDF.select('doc', split(reviewListDF.doc, " ").alias("feature"))
  if ngram >1:
    nGramDF = NGram(n=ngram,inputCol = 'feature',outputCol='grams').transform(nGramDF)
    nGramDF = nGramDF.drop('doc')
    nGramDF = nGramDF.withColumn("doc_id", monotonically_increasing_id())
    nGramDF = nGramDF.withColumn("token", explode(nGramDF.grams))
  else:
    nGramDF = nGramDF.drop('doc')
    nGramDF = nGramDF.withColumn("doc_id", monotonically_increasing_id())
    nGramDF = nGramDF.withColumn("token", explode(nGramDF.feature))
  return nGramDF




IndentationError: unindent does not match any outer indentation level (<tokenize>, line 9)

In [15]:
monoGramDF = nGram_DF_Creating(1,reviewDF)


UnboundLocalError: local variable 'nGramDF' referenced before assignment

In [16]:
from nltk import ngrams
from pyspark.sql.functions import split
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.ml.feature import NGram
from pyspark.sql.functions import explode

def nGram_DF_Creating(n_num,reviewListDF):
    nGramDF = reviewListDF.select('doc', split(reviewListDF.doc, " ").alias("feature"))
  if n_num >1:
    nGramDF = NGram(n=n_num,inputCol = 'feature',outputCol='grams').transform(nGramDF)
    nGramDF = nGramDF.drop('doc')
    nGramDF = nGramDF.withColumn("doc_id", monotonically_increasing_id())
    nGramDF = nGramDF.withColumn("token", explode(nGramDF.grams))
  else:
    nGramDF = nGramDF.drop('doc')
    nGramDF = nGramDF.withColumn("doc_id", monotonically_increasing_id())
    nGramDF = nGramDF.withColumn("token", explode(nGramDF.feature))
  return nGramDF




IndentationError: unindent does not match any outer indentation level (<tokenize>, line 9)

In [18]:
monoGramDF = nGram_DF_Creating(1,reviewDF)


UnboundLocalError: local variable 'nGramDF' referenced before assignment

In [19]:
from nltk import ngrams
from pyspark.sql.functions import split
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.ml.feature import NGram
from pyspark.sql.functions import explode

def nGram_DF_Creating(n_num,reviewListDF):
  nGramDF = reviewListDF.select('doc', split(reviewListDF.doc, " ").alias("feature"))
  if n_num >1:
    nGramDF = NGram(n=n_num,inputCol = 'feature',outputCol='grams').transform(nGramDF)
    nGramDF = nGramDF.drop('doc')
    nGramDF = nGramDF.withColumn("doc_id", monotonically_increasing_id())
    nGramDF = nGramDF.withColumn("token", explode(nGramDF.grams))
  else:
    nGramDF = nGramDF.drop('doc')
    nGramDF = nGramDF.withColumn("doc_id", monotonically_increasing_id())
    nGramDF = nGramDF.withColumn("token", explode(nGramDF.feature))
  return nGramDF




In [20]:
monoGramDF = nGram_DF_Creating(1,reviewDF)


In [21]:
monoGram_summary = dt_idf_DF_Create(monoGramDF,reviewDF)





AttributeError: 'DataFrame' object has no attribute '_get_object_id'

In [22]:
monoGram_summary = dt_idf_DF_Create(monoGramDF,reviewDF)





AttributeError: 'DataFrame' object has no attribute '_get_object_id'

In [23]:
monoGramDF = nGram_DF_Creating(1,reviewDF)
monoGramDF.head()

Row(feature=['arrived', 'broken', 'manufacturer', 'defect', 'two', 'of', 'the', 'legs', 'of', 'the', 'base', 'were', 'not', 'completely', 'formed', 'so', 'there', 'was', 'no', 'way', 'to', 'insert', 'the', 'casters', 'i', 'unpackaged', 'the', 'entire', 'chair', 'and', 'hardware', 'before', 'noticing', 'this', 'so', 'ill', 'spend', 'twice', 'the', 'amount', 'of', 'time', 'boxing', 'up', 'the', 'whole', 'useless', 'thing', 'and', 'send', 'it', 'back', 'with', 'a', '1star', 'review', 'of', 'part', 'of', 'a', 'chair', 'i', 'never', 'got', 'to', 'sit', 'in', 'i', 'will', 'go', 'so', 'far', 'as', 'to', 'include', 'a', 'picture', 'of', 'what', 'their', 'injection', 'molding', 'and', 'quality', 'assurance', 'process', 'missed', 'though', 'i', 'will', 'be', 'hesitant', 'to', 'buy', 'again', 'it', 'makes', 'me', 'wonder', 'if', 'there', 'arent', 'missing', 'structures', 'and', 'supports', 'that', 'dont', 'impede', 'the', 'assembly', 'process'], doc_id=0, token='arrived')

In [24]:
monoGramDF.show()


+--------------------+------+------------+
|             feature|doc_id|       token|
+--------------------+------+------------+
|[arrived, broken,...|     0|     arrived|
|[arrived, broken,...|     0|      broken|
|[arrived, broken,...|     0|manufacturer|
|[arrived, broken,...|     0|      defect|
|[arrived, broken,...|     0|         two|
|[arrived, broken,...|     0|          of|
|[arrived, broken,...|     0|         the|
|[arrived, broken,...|     0|        legs|
|[arrived, broken,...|     0|          of|
|[arrived, broken,...|     0|         the|
|[arrived, broken,...|     0|        base|
|[arrived, broken,...|     0|        were|
|[arrived, broken,...|     0|         not|
|[arrived, broken,...|     0|  completely|
|[arrived, broken,...|     0|      formed|
|[arrived, broken,...|     0|          so|
|[arrived, broken,...|     0|       there|
|[arrived, broken,...|     0|         was|
|[arrived, broken,...|     0|          no|
|[arrived, broken,...|     0|         way|
+----------

In [25]:
monoGramDF = nGram_DF_Creating(2,reviewDF)


In [26]:
monoGramDF.show()


+--------------------+--------------------+------+-------------------+
|             feature|               grams|doc_id|              token|
+--------------------+--------------------+------+-------------------+
|[arrived, broken,...|[arrived broken, ...|     0|     arrived broken|
|[arrived, broken,...|[arrived broken, ...|     0|broken manufacturer|
|[arrived, broken,...|[arrived broken, ...|     0|manufacturer defect|
|[arrived, broken,...|[arrived broken, ...|     0|         defect two|
|[arrived, broken,...|[arrived broken, ...|     0|             two of|
|[arrived, broken,...|[arrived broken, ...|     0|             of the|
|[arrived, broken,...|[arrived broken, ...|     0|           the legs|
|[arrived, broken,...|[arrived broken, ...|     0|            legs of|
|[arrived, broken,...|[arrived broken, ...|     0|             of the|
|[arrived, broken,...|[arrived broken, ...|     0|           the base|
|[arrived, broken,...|[arrived broken, ...|     0|          base were|
|[arri

In [27]:
monoGramDF = nGram_DF_Creating(1,reviewDF)


In [28]:
monoGramDF.show()


+--------------------+------+------------+
|             feature|doc_id|       token|
+--------------------+------+------------+
|[arrived, broken,...|     0|     arrived|
|[arrived, broken,...|     0|      broken|
|[arrived, broken,...|     0|manufacturer|
|[arrived, broken,...|     0|      defect|
|[arrived, broken,...|     0|         two|
|[arrived, broken,...|     0|          of|
|[arrived, broken,...|     0|         the|
|[arrived, broken,...|     0|        legs|
|[arrived, broken,...|     0|          of|
|[arrived, broken,...|     0|         the|
|[arrived, broken,...|     0|        base|
|[arrived, broken,...|     0|        were|
|[arrived, broken,...|     0|         not|
|[arrived, broken,...|     0|  completely|
|[arrived, broken,...|     0|      formed|
|[arrived, broken,...|     0|          so|
|[arrived, broken,...|     0|       there|
|[arrived, broken,...|     0|         was|
|[arrived, broken,...|     0|          no|
|[arrived, broken,...|     0|         way|
+----------

In [29]:
monoGram_summary = dt_idf_DF_Create(monoGramDF,reviewDF)





AttributeError: 'DataFrame' object has no attribute '_get_object_id'

In [30]:
from nltk import ngrams
from pyspark.sql.functions import split
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.ml.feature import NGram
from pyspark.sql.functions import explode

def nGram_DF_Create(n_num,reviewListDF):
  nGramDF = reviewListDF.select('doc', split(reviewListDF.doc, " ").alias("feature"))
  if n_num >1:
    nGramDF = NGram(n=n_num,inputCol = 'feature',outputCol='grams').transform(nGramDF)
    nGramDF = nGramDF.drop('doc')
    nGramDF = nGramDF.withColumn("doc_id", monotonically_increasing_id())
    nGramDF = nGramDF.withColumn("token", explode(nGramDF.grams))
  else:
    nGramDF = nGramDF.drop('doc')
    nGramDF = nGramDF.withColumn("doc_id", monotonically_increasing_id())
    nGramDF = nGramDF.withColumn("token", explode(nGramDF.feature))
  return nGramDF




In [31]:

import numpy as np
from pyspark.sql.functions import col
from pyspark.sql.functions import log

def dt_idf_DF_Create(input_df,reviewListDF):
      
    totalcount = reviewListDF.count()

    input_df.createOrReplaceTempView("input_df")
    tfreq = spark.sql("SELECT doc_id,token,COUNT(feature) AS tf FROM input_df GROUP BY doc_id, token")
    dfreq = spark.sql("SELECT token,COUNT(DISTINCT feature) AS dfcount FROM input_df GROUP BY token")
 

    dfreq = dfreq.withColumn("idf",log(totalcount/(dfreq.dfcount+1)))
    dfreq = dfreq.withColumn("idf",log(totalcount/(dfreq.dfcount+1)))


    tfreq.createOrReplaceTempView("tfreq")
    dfreq.createOrReplaceTempView("dfreq")
    tf_idf = spark.sql("SELECT tfreq.token, tfreq.doc_id, tfreq.tf,dfreq.dfcount ,dfreq.idf FROM dfreq LEFT JOIN tfreq ON tfreq.token = dfreq.token")
    tf_idf = tf_idf.withColumn('tf_idf_stats',tf_idf.tf * tf_idf.idf)

    return tf_idf



In [32]:
monoGram_summary = dt_idf_DF_Create(monoGramDF,reviewDF)





In [33]:
monoGram_summary.show()

+--------+-----------+---+-------+------------------+------------------+
|   token|     doc_id| tf|dfcount|               idf|      tf_idf_stats|
+--------+-----------+---+-------+------------------+------------------+
|      07|      48131|  1|     16|10.066006482033902|10.066006482033902|
|      07|42949701438|  1|     16|10.066006482033902|10.066006482033902|
|      07| 8589964025|  1|     16|10.066006482033902|10.066006482033902|
|      07|17179905456|  1|     16|10.066006482033902|10.066006482033902|
|      07|      40722|  1|     16|10.066006482033902|10.066006482033902|
|      07| 8589964024|  1|     16|10.066006482033902|10.066006482033902|
|      07|25769817402|  1|     16|10.066006482033902|10.066006482033902|
|      07|      53372|  1|     16|10.066006482033902|10.066006482033902|
|      07|51539621343|  1|     16|10.066006482033902|10.066006482033902|
|      07|      38468|  1|     16|10.066006482033902|10.066006482033902|
|      07|17179878238|  1|     16|10.06600648203390

In [34]:
biGramDF = nGram_DF_Create(2,reviewDF)
biGram_summary = dt_idf_DF_Create(biGramDF,reviewDF)




In [35]:
biGram_summary.show()

+-----+-----------+---+-------+------------------+------------------+
|token|     doc_id| tf|dfcount|               idf|      tf_idf_stats|
+-----+-----------+---+-------+------------------+------------------+
|   51| 8589938096|  1|      2| 11.80060753742201| 11.80060753742201|
|   51|42949721080|  1|      2| 11.80060753742201| 11.80060753742201|
|  air|17179875666|  1|      4|11.289781913656018|11.289781913656018|
|  air|      19134|  1|      4|11.289781913656018|11.289781913656018|
|  air|25769838120|  1|      4|11.289781913656018|11.289781913656018|
|  air|25769838933|  1|      4|11.289781913656018|11.289781913656018|
|  bad|       3612|  1|     26|  9.60338296008579|  9.60338296008579|
|  bad|42949716535|  1|     26|  9.60338296008579|  9.60338296008579|
|  bad|42949677175|  1|     26|  9.60338296008579|  9.60338296008579|
|  bad|17179881174|  1|     26|  9.60338296008579|  9.60338296008579|
|  bad|17179896179|  1|     26|  9.60338296008579|  9.60338296008579|
|  bad|60129576637| 

In [36]:
triGramDF = nGram_DF_Create(2,reviewDF)
triGram_summary = dt_idf_DF_Create(triGramDF,reviewDF)



In [37]:
triGram_summary.show()



+-----+-----------+---+-------+------------------+------------------+
|token|     doc_id| tf|dfcount|               idf|      tf_idf_stats|
+-----+-----------+---+-------+------------------+------------------+
|   51| 8589938096|  1|      2| 11.80060753742201| 11.80060753742201|
|   51|42949721080|  1|      2| 11.80060753742201| 11.80060753742201|
|  air|17179875666|  1|      4|11.289781913656018|11.289781913656018|
|  air|      19134|  1|      4|11.289781913656018|11.289781913656018|
|  air|25769838120|  1|      4|11.289781913656018|11.289781913656018|
|  air|25769838933|  1|      4|11.289781913656018|11.289781913656018|
|  bad|       3612|  1|     26|  9.60338296008579|  9.60338296008579|
|  bad|42949716535|  1|     26|  9.60338296008579|  9.60338296008579|
|  bad|42949677175|  1|     26|  9.60338296008579|  9.60338296008579|
|  bad|17179881174|  1|     26|  9.60338296008579|  9.60338296008579|
|  bad|17179896179|  1|     26|  9.60338296008579|  9.60338296008579|
|  bad|60129576637| 

In [38]:
triGram_summary.write.csv('triGram_summary.csv')

Py4JJavaError: An error occurred while calling o334.csv.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:226)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:178)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:108)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:106)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:131)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$execute$1(SparkPlan.scala:175)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeQuery$1(SparkPlan.scala:213)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:210)
	at or

In [39]:
biGram_summary.write.csv('biGram_summary.csv')

Py4JJavaError: An error occurred while calling o338.csv.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:226)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:178)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:108)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:106)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:131)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$execute$1(SparkPlan.scala:175)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeQuery$1(SparkPlan.scala:213)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:210)
	at or

In [40]:
monoGram_summary.write.csv('monoGram_summary.csv')

Py4JJavaError: An error occurred while calling o347.csv.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:226)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:178)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:108)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:106)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:131)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$execute$1(SparkPlan.scala:175)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeQuery$1(SparkPlan.scala:213)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:210)
	at or

In [41]:
breakpoint()
from pyspark.sql import SparkSession
import numpy as py
from pyspark.sql.types import Row
import pyspark.sql.functions as fn 
